In [2]:
# imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

pd.options.plotting.backend = "plotly"

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet import Prophet
import itertools
from sktime.performance_metrics.forecasting import (
    mean_absolute_scaled_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
)
import time
import holidays
import datetime


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/utils/validation/series.py:22: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# define functions

In [3]:
# preprocessing_endo
def preprocessing_endo(ts, horizon, split=True):
    fb_df = ts[["sales"]]
    fb_df["sales"] = ts["sales"] / 1e6
    fb_df = fb_df.reset_index().rename({"date": "ds", "sales": "y"}, axis=1)

    if split:
        fb_train = fb_df.iloc[:-horizon]
        fb_test = fb_df.iloc[-horizon:]
        return {"y_train": fb_train, "y_test": fb_test}
    else:
        return fb_df


In [ ]:
# preprocessing_exog
def preprocessing_exog(df_exog):
    df_promo = pd.DataFrame({
        "holiday": "promo_day",
        "ds": df_exog[df_exog["promo_day"] == 1]["promo_day"].index})
    df_off = pd.DataFrame({
        "holiday": "off_day",
        "ds": df_exog[df_exog["off_day"] == 1]["off_day"].index})
    return pd.concat([df_promo, df_off])


In [19]:
# grid seach Prophet
def grid_search_prophet(prophet_data_endo, prophet_data_exog, param_grid):
    # create df to store result
    mape_PROPHET = pd.DataFrame(columns=[
        "changepoint_prior_scale", 
        "seasonality_prior_scale",
        "holidays_prior_scale",
        "mape",
        "mae",
        "rmse",
        ])
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

    # grid search
    y_train = prophet_data_endo['y_train']
    y_test = prophet_data_endo['y_test']
    for params in all_params:
        m = Prophet(holidays=prophet_data_exog,
                    **params).fit(y_train)  # Fit model with given params
        y_hat = m.predict(y_test)['yhat']
        mape_PROPHET = mape_PROPHET.append(
            {"changepoint_prior_scale": params["changepoint_prior_scale"], 
             "seasonality_prior_scale": params["seasonality_prior_scale"], 
             "holidays_prior_scale": params["holidays_prior_scale"], 
             "mape": round(mean_absolute_percentage_error(y_test['y'], y_hat), 3),
             "mae": round(mean_absolute_error(y_test['y'], y_hat), 3),
             "rmse": round(mean_squared_error(y_test['y'], y_hat, square_root=True), 3),
             }, ignore_index=True)
    # mape_PROPHET.sort_values('mape', inplace=True)
    return mape_PROPHET


In [ ]:
# CV 
def cross_validation_result(data, exog, params, model_name, horizon, rolls=4):
    """ """
    mae_CVs = []
    rmse_CVs = []
    mape_CVs = []
    mase_CVs = []
    for i in range(rolls):
        print(f"fold {i}---------------")
        
        # split data
        y_train = data.iloc[: -(rolls - i) * horizon]
        y_test = data.iloc[
            np.r_[-(rolls - i) * horizon : -(rolls - i - 1) * horizon]]

        # fit model
        model = Prophet(**params, holidays=exog)
        model.fit(y_train)
        
        # make forecast
        y_hat = model.predict(y_test)['y_hat']

        # score
        mae_CVs.append(round(mean_absolute_error(y_test['y'], y_hat), 3))
        rmse_CVs.append(round(mean_squared_error(y_test['y'], y_hat, square_root=True), 3))
        mape_CVs.append(round(mean_absolute_percentage_error(y_test['y'], y_hat), 3))
        mase_CVs.append(round(mean_absolute_scaled_error(y_test['y'], y_hat, y_train=y_train['y']), 3))
    
    return {'store':model_name,
           'mae_PROPHET':np.mean(mae_CVs),
           'rmse_PROPHET':np.mean(rmse_CVs),
           'mape_PROPHET':np.mean(mape_CVs),
           'mase_PROPHET':np.mean(mase_CVs),
           'fc_PROPHET':y_hat,
           }


# read data

In [4]:
df_store = pd.read_pickle("data/df_daily.pkl")
df_company = df_store.groupby("date").sum()[["sales"]]
df_exog = pd.read_pickle("data/df_exog.pkl")


# horizon = 7

## tune on company data

In [5]:
horizon = 7
df_company_pro = preprocessing_endo(df_company, horizon, split=True)
df_exog_pro = preprocessing_exog(df_exog)


In [20]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.001, 0.01, 0.05, 0.1, 0.5], 
    "seasonality_prior_scale": [0.01, 0.1, 1, 10],
    "holidays_prior_scale": [0.01, 0.1, 1, 10],
    }

grid_search_result = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2683.31   3.11993e-05       5191.88      0.3629           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     106       2685.34   0.000236981       5576.32   4.755e-08       0.001      172  LS failed, Hessian reset 
     190       2686.38   9.87338e-09       4874.52      0.6034      0.6034      271   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      92       2732.77   0.000153759       4865.87    3.33e-08       0.001      154  LS failed, Hessian reset 
      99       2733.22   1.53875e-05       4813.37      0.1784           1      163   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     133       2733.45   1.58682e-06       4471.56   3.346e-10       0.001      246  LS failed, Hessian reset 
     152       2733.46   6.14168e-09       3959.31      0.2564      0.2564      268   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2819.68    0.00295478       101.845      0.3069      0.3069      119   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2821.77     0.0019403       25.8795           1           1      239   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2822.38     0.0004924       22.4665      0.9938      0.9938      357   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2822.83   8.30955e-05       6.22544      0.1817           1      477   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       2823.03   0.000986215       15.5377      0.8148      0.8148      590   
    Iter      log prob        ||dx||      ||grad||       alpha  

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2819.55    0.00325933       43.7645           1           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2821.38    0.00206414       58.6002     0.07598           1      232   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2822.27   0.000787447       35.9459      0.8456      0.8456      346   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2822.75   0.000486484       27.0489           1           1      467   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       2823.09    0.00140344       55.9312      0.5644      0.5644      585   
    Iter      log prob        ||dx||      ||grad||       alpha  

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

In [46]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.05], 
    "seasonality_prior_scale": [10],
    "holidays_prior_scale": [10],
    }

grid_search_result1 = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -15.2228
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2803.75   0.000219649       68.7782      0.7949      0.7949      140   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     126       2804.01     0.0010467       151.622   1.005e-05       0.001      219  LS failed, Hessian reset 
     199       2804.15   1.04472e-05       69.6038           1           1      313   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2804.17   1.57902e-05       78.2213      0.4984      0.4984      433   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2804.21   1.12076e-06       60.7159      0.2379           1      563   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     415       2804.21   6.26992e-06    

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/sktime/performance_metrics/forecasting/_functions.py:1543: FutureWarning: In the percentage error metric functions the default argument symmetric=True is changing to symmetric=False in v0.12.0.
  warn(
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_

In [42]:
grid_search_result.sort_values(['mape', 'rmse'], inplace=True)
# grid_search_result.head(10).to_csv("results/f8/PROPHET_param_7.csv")
grid_search_result#.head(10)


,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,mape,mae,rmse
44,0.1,10.0,0.01,0.1,113.516,177.69
36,0.1,0.1,0.01,0.1,113.312,177.961
40,0.1,1.0,0.01,0.1,113.006,178.123
32,0.1,0.01,0.01,0.101,114.927,180.84
8,0.001,1.0,0.01,0.11,124.677,180.575
...,...,...,...,...,...,...
27,0.01,1.0,10.0,0.174,162.259,195.028
18,0.01,0.01,1.0,0.175,162.689,198.234
26,0.01,1.0,1.0,0.176,163.5,195.8
19,0.01,0.01,10.0,0.176,162.82,198.553


In [47]:
# grid_search_result.query('changepoint_prior_scale==10')
#['seasonality_prior_scale', 'holidays_prior_scale', 'changepoint_prior_scale']
grid_search_result1

,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,mape,mae,rmse
0,0.05,10.0,10.0,0.148,148.398,188.177


In [38]:
# grid_search_result.to_pickle("results/f8/PROPHET_param_7.pkl")
# grid_search_result.to_csv("results/f8/PROPHET_param_7.csv")
# grid_search_result = pd.read_pickle("results/f8/PROPHET_param_7.pkl")
tuned_mape = grid_search_result.iloc[0, 3]
tuned_prophet_params = grid_search_result.iloc[0,:3].to_dict()

tuned_mape, tuned_prophet_params


(0.1,
 {'changepoint_prior_scale': 0.1,
  'seasonality_prior_scale': 10.0,
  'holidays_prior_scale': 0.01})

## fit on store data

In [1]:
all_stores_result_CV = []
for store in df_store["store_id"].unique():#[:2]:
    print(f"processing stores {store}...\n")
    model_name = "store_" + str(store)

    # data
    df_1_store = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_1_store_pro = preprocessing_endo(df_1_store, horizon, split=False)

    # define model
    # model_store = Prophet(**tuned_prophet_params, holidays=df_exog_pro)
    
    # score model with CV on store data
    cv_score = cross_validation_result(df_1_store_pro, df_exog_pro, tuned_prophet_params, model_name=model_name)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


NameError: name 'df_store' is not defined

## result

In [320]:
# all_stores_result_CV.to_pickle("results/f8/PROPHET_result_7.pkl")


# horizon = 14

## tune on company data

In [323]:
horizon = 14
df_company_pro = preprocessing_endo(df_company, horizon, split=True)
df_exog_pro = preprocessing_exog(df_exog)


In [324]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.005, 0.01, 0.05, 0.1],
    "seasonality_prior_scale": [1.0, 0.01, 0.1, 1.0],
    "daily_seasonality": [False],
    "weekly_seasonality": [True],
    "yearly_seasonality": [True],
}

grid_search_result = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -12.6088
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2755.85   0.000125592       803.786     0.06238           1      149   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     129       2755.96   8.98668e-06       813.634   1.245e-08       0.001      226  LS failed, Hessian reset 
     153       2755.97   8.53486e-07       695.591    1.23e-09       0.001      292  LS failed, Hessian reset 
     165       2755.97   1.62217e-08        564.37      0.4354           1      307   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -12.6088
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2734.51    1.3616e-05         568.8      0.4873      0.4873      139   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2735.72   7.67497e-05       458.781      0.3309           1      272   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     254       2735.81   1.36879e-05       750.343   2.192e-08       0.001      428  LS failed, Hessian reset 
     299          2736   8.41739e-06       707.174           1           1      483   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     343       2736.01   2.35892e-06       622.305   3.063e-09       0.001      584  LS failed, Hessian reset 
     362       2736.01   9.47511e-10       602.804     0.02435     0.02435      611   
Optimization terminat

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -12.6088
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2794.26   0.000886704       100.555      0.4455           1      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     160        2794.9   0.000159685       70.6412   3.959e-06       0.001      243  LS failed, Hessian reset 
     199          2795   0.000207348       32.5902      0.7266      0.7266      292   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        2795.2   0.000325161       33.9026      0.7023      0.7023      415   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     334       2795.29   0.000223414       43.3664   5.563e-06       0.001      509  LS failed, Hessian reset 
     399       2795.33   2.12325e-05       36.5292           1           1      596   
    Iter      log pro

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -12.6088
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2794.56    0.00155807       49.9909           1           1      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2795.26   8.49244e-05       36.0022           1           1      246   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     200       2795.26   1.01362e-05       34.4905   2.815e-07       0.001      289  LS failed, Hessian reset 
     220       2795.26   4.66566e-07       38.3147      0.3297      0.3297      312   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(


In [333]:
# grid_search_result.to_pickle("results/f8/PROPHET_param_14.pkl")
# grid_search_result = pd.read_pickle("results/f8/PROPHET_param_14.pkl")
tuned_prophet_params = grid_search_result.iloc[0, 0]
tuned_mape = grid_search_result.iloc[0, 1]


## fit on store data

In [326]:
all_stores_result_CV = []
for store in df_store["store_id"].unique():#[:2]:
    print(f"processing stores {store}...\n")
    model_name = "store_" + str(store)

    # data
    df_1_store = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_1_store_pro = preprocessing_endo(df_1_store, horizon, split=False)

    # define model
    # model_store = Prophet(**tuned_prophet_params, holidays=df_exog_pro)
    
    # score model with CV on store data
    cv_score = cross_validation_result(df_1_store_pro, df_exog_pro, tuned_prophet_params, model_name=model_name)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


processing stores 307222...

fold 0---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -7.38369
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2759.91   7.61095e-05       49.7687       0.404       0.404      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     138       2760.22   0.000168839       66.5303   4.704e-06       0.001      228  LS failed, Hessian reset 
     199        2760.3   0.000129406       31.2291      0.7973      0.7973      300   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     275       2760.34     5.129e-07       26.1187      0.2196      0.8885      408   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


fold 1---------------


...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


fold 3---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -17.4505
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1376.65     0.0013799       43.8279      0.7903      0.7903      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     143       1378.26   0.000450499        46.174   4.033e-06       0.001      213  LS failed, Hessian reset 
     199       1378.75   5.85531e-06       37.6958           1           1      285   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       1379.27   0.000145741       42.3985           1           1      410   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     388       1379.59   0.000347339       38.3813   8.456e-06       0.001      602  LS failed, Hessian reset 
     399       1379.63   0.000121736       37.0774      0.4652      0.4652      616   
    Iter      log pro

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


## result

In [327]:
all_stores_result_CV.describe()


,mae_PROPHET,rmse_PROPHET,mape_PROPHET,mase_PROPHET
count,38.000000,38.000000,38.000000,38.000000
mean,7.010829,8.870763,0.338375,0.951572
std,5.261995,7.191575,0.098701,0.203832
min,2.873500,3.770750,0.234000,0.588000
25%,3.747187,4.523313,0.263375,0.808187
50%,5.513500,6.928500,0.309500,0.917000
75%,7.431750,9.393188,0.380937,1.064188
max,27.242250,36.934000,0.647000,1.539000


In [334]:
# all_stores_result_CV.to_pickle("results/f8/PROPHET_result_14.pkl")


# horizon = 21
## tune on company data

In [336]:
horizon = 21
df_company_pro = preprocessing_endo(df_company, horizon, split=True)
df_exog_pro = preprocessing_exog(df_exog)


In [337]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.005, 0.01, 0.05, 0.1],
    "seasonality_prior_scale": [1.0, 0.01, 0.1, 1.0],
    "daily_seasonality": [False],
    "weekly_seasonality": [True],
    "yearly_seasonality": [True],
}

grid_search_result = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -12.6377
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2737.85    0.00059926       711.864           1           1      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     146       2738.66   0.000125301       762.158   1.573e-07       0.001      233  LS failed, Hessian reset 
     190        2738.8   8.60801e-06        669.61   1.009e-08       0.001      324  LS failed, Hessian reset 
     199        2738.8   2.63835e-08       648.459     0.01787      0.2239      339   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     220        2738.8   4.28414e-09       680.194       0.327       0.327      372   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -12.6377
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2718.74   1.26231e-05       638.585           1           1      137   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110       2719.06    0.00036706       538.154   3.671e-07       0.001      200  LS failed, Hessian reset 
     162        2719.2    3.9278e-07       634.656   6.387e-10       0.001      312  LS failed, Hessian reset 
     171        2719.2     8.938e-09       583.498       0.395           1      326   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -12.6377
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2776.96    0.00127522       69.2042           1           1      119   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     102       2777.01   0.000111052       44.4384     1.3e-06       0.001      171  LS failed, Hessian reset 
     137       2777.28   0.000276751       115.075   3.729e-06       0.001      263  LS failed, Hessian reset 
     199       2777.67     0.0135349       53.1534           1           1      335   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299          2778   0.000217596       33.4996           1           1      454   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     353       2778.05   7.42335e-05       40.2354   1.677e-06       0.001      574  LS failed, Hessian rese

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -12.6377
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2777.38   0.000650045       66.1093      0.5527           1      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2778.07   8.28763e-05       42.9295      0.4257           1      254   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     288       2778.21   0.000128011       42.2271   3.159e-06       0.001      414  LS failed, Hessian reset 
     299       2778.21   7.62074e-05       26.1917      0.7427      0.7427      426   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2778.27    0.00014366       38.2238           1           1      545   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     440       2778.27   1.54815e-06    

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(


In [338]:
grid_search_result.to_pickle("results/f8/PROPHET_param_21.pkl")
# grid_search_result = pd.read_pickle("results/f8/PROPHET_param_14.pkl")
tuned_prophet_params = grid_search_result.iloc[0, 0]
tuned_mape = grid_search_result.iloc[0, 1]


## fit on store data

In [339]:
all_stores_result_CV = []
for store in df_store["store_id"].unique():#[:2]:
    print(f"processing stores {store}...\n")
    model_name = "store_" + str(store)

    # data
    df_1_store = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_1_store_pro = preprocessing_endo(df_1_store, horizon, split=False)

    # define model
    # model_store = Prophet(**tuned_prophet_params, holidays=df_exog_pro)
    
    # score model with CV on store data
    cv_score = cross_validation_result(df_1_store_pro, df_exog_pro, tuned_prophet_params, model_name=model_name)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


processing stores 307222...

fold 0---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -7.38369
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      74        2746.8   0.000450462       650.258   6.854e-07       0.001      122  LS failed, Hessian reset 
      99          2747   1.87493e-05        693.85      0.7765      0.7765      152   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     162       2748.79   6.05385e-05       725.969   1.115e-07       0.001      267  LS failed, Hessian reset 
     199       2748.91   8.95408e-06       651.013   1.079e-08       0.001      346  LS failed, Hessian reset 
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     260       2748.93   1.12011e-05       614.639   1.371e-08       0.001      477  LS failed, Hessian reset 
     299       2748.93   2.44046e-07       730.753       0.718       0.718      523   
    Iter      log prob        ||dx||      ||grad||       alpha   

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


fold 1---------------


...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


fold 3---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -17.4505
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1413.05   2.08857e-06       1045.89     0.07164           1      126   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       1415.64   5.09412e-07         693.5           1           1      256   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     215       1415.64   9.94749e-09       764.514      0.2515      0.2515      276   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


## result

In [340]:
all_stores_result_CV.to_pickle("results/f8/PROPHET_result_21.pkl")


# horizon = 28
## tune on company data

In [341]:
horizon = 28
df_company_pro = preprocessing_endo(df_company, horizon, split=True)
df_exog_pro = preprocessing_exog(df_exog)


In [342]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.005, 0.01, 0.05, 0.1],
    "seasonality_prior_scale": [1.0, 0.01, 0.1, 1.0],
    "daily_seasonality": [False],
    "weekly_seasonality": [True],
    "yearly_seasonality": [True],
}

grid_search_result = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -8.9084
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        2721.3   7.29113e-05       742.337      0.9146      0.9146      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     105       2721.84    0.00026014       564.822   4.409e-07       0.001      174  LS failed, Hessian reset 
     161        2722.1   8.26751e-06       772.229   1.361e-08       0.001      298  LS failed, Hessian reset 
     199        2722.1   2.30836e-08       691.044      0.2431           1      349   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -8.9084
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2700.88   0.000950329       704.535      0.5564      0.5564      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2702.45   0.000600592       602.043      0.9261      0.9261      268   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     228        2702.7   3.13523e-05       625.666   4.513e-08       0.001      339  LS failed, Hessian reset 
     299       2702.89   1.11392e-07       555.871      0.3367      0.3367      433   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     302       2702.89   3.69225e-09       488.866     0.06774     0.06774      440   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -8.9084
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2760.45    0.00033987       38.4388      0.2755           1      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2761.21    0.00149693       68.0704           1           1      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2761.44   9.13805e-06       33.5699      0.9881      0.9881      358   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     312       2761.44   1.04638e-06       35.2831      0.2091           1      376   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/

Initial log joint probability = -8.9084
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2760.65   0.000920255       97.3873           1           1      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     120        2761.1   0.000146546       70.1836   3.407e-06       0.001      188  LS failed, Hessian reset 
     156       2761.34   5.18265e-05       30.3218   1.666e-06       0.001      264  LS failed, Hessian reset 
     199       2761.35   0.000362034       31.8955      0.6764           1      322   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     234       2761.51   0.000248233       102.144   3.804e-06       0.001      410  LS failed, Hessian reset 
     299       2761.66     6.902e-05        38.964       0.773       0.773      496   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/150540093.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(


In [343]:
grid_search_result.to_pickle("results/f8/PROPHET_param_28.pkl")
# grid_search_result = pd.read_pickle("results/f8/PROPHET_param_14.pkl")
tuned_prophet_params = grid_search_result.iloc[0, 0]
tuned_mape = grid_search_result.iloc[0, 1]


## fit on store data

In [344]:
all_stores_result_CV = []
for store in df_store["store_id"].unique():#[:2]:
    print(f"processing stores {store}...\n")
    model_name = "store_" + str(store)

    # data
    df_1_store = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_1_store_pro = preprocessing_endo(df_1_store, horizon, split=False)

    # define model
    # model_store = Prophet(**tuned_prophet_params, holidays=df_exog_pro)
    
    # score model with CV on store data
    cv_score = cross_validation_result(df_1_store_pro, df_exog_pro, tuned_prophet_params, model_name=model_name)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


processing stores 307222...

fold 0---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -7.38369
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2747.74   0.000812143       731.259   1.124e-06       0.001      169  LS failed, Hessian reset 
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2748.06   2.24024e-07       767.606           1           1      307   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     262       2748.83   3.27431e-05       701.355   3.644e-08       0.001      429  LS failed, Hessian reset 
     295       2748.83   6.99533e-07       692.088   8.036e-10       0.001      516  LS failed, Hessian reset 
     299       2748.84   3.62054e-08        579.42      0.3017           1      522   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     307       2748.84   8.18073e-09       468.498      0.2626      0.2626      532 

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


fold 1---------------


...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


fold 3---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -17.4505
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      87       1413.29   0.000188337       997.431   1.956e-07       0.001      142  LS failed, Hessian reset 
      99        1413.4   5.14364e-06       920.056      0.3476      0.3476      156   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     122       1413.49   5.28107e-05       913.257   4.759e-08       0.001      227  LS failed, Hessian reset 
     170        1413.6   2.10426e-05       1003.08   2.386e-08       0.001      334  LS failed, Hessian reset 
     199       1413.65   4.86355e-07       985.914           1           1      371   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     223       1413.65   5.47143e-09       815.015     0.06978           1      402   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was 

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


## result

In [ ]:
all_stores_result_CV.to_pickle("results/f8/PROPHET_result_28.pkl")
